In [ ]:
# NEEDS WRADLIB 1.19 !! 

import wradlib as wrl
import numpy as np
import sys
import glob
import xarray as xr
import os
import datetime as dt
import pandas as pd
from tqdm.notebook import trange, tqdm

import warnings
warnings.filterwarnings('ignore')
import xradar as xd
import datatree

In [ ]:
import netCDF4
import packaging

In [ ]:
assert packaging.version.Version(netCDF4.__version__) <=  packaging.version.Version("1.6.0")

In [ ]:
print(f"xradar: {xd.__version__}")
print(f"wradlib: {wrl.__version__}")

In [ ]:
import time
start_time = time.time()

# Read DWD file to retrieve encoding values

In [ ]:
dwd = xr.open_dataset("/automount/ags/jgiles/turkey_test/ras07-vol5minng01_sweeph5onem_allmoms_00-2017072700005800-pro-10392-hd5", group="sweep_0")
display(dwd)

# tamper with encoding

In [ ]:
drop = ["szip", "zstd", "source", "chunksizes", "bzip2", "blosc", "shuffle", "fletcher32", "original_shape", "coordinates", "contiguous"]
dwd_enc = {k: {key: v.encoding[key] for key in v.encoding if key not in drop} for k, v in dwd.data_vars.items() if v.ndim == 3}
dwd_enc["PHIDP"] = dwd_enc["UPHIDP"]
dwd_enc["DBTH"] = dwd_enc["TH"]
dwd_enc["DBTV"] = dwd_enc["TV"]
dwd_enc


# Import and set Dask stuff

In [ ]:
import dask
from dask.distributed import Client
# not sure if this is needed
client = Client(n_workers=8)
client
from dask.diagnostics import ProgressBar

# Get Files

In [ ]:
# Get all files for one day
htypath = sorted(glob.glob("/automount/ags/jgiles/turkey_test/acq/OLDDATA/uza/RADAR/2017/07/27/HTY/RAW/*"))


In [ ]:
# Create a dataframe to store the metadata of all files and then select it more easily

# Read attributes of files
radarid = []
dtime = []
taskname = []
elevation = []
nrays_expected = []
nrays_written = []
nbins = []
rlastbin = []
binlength = []
horbeamwidth = []
fpath = []

for f in htypath:
    print(".", end="")
    # Read metadata
    m = wrl.io.read_iris(f, loaddata=False, keep_old_sweep_data=True)
    # Extract info
    fname = os.path.basename(f).split(".")[0]
    radarid_ = fname[0:3]
    dtimestr = fname[3:]
    dtime_ = dt.datetime.strptime(dtimestr, "%y%m%d%H%M%S")
    taskname_ = m["product_hdr"]["product_configuration"]["task_name"].strip()
    nbins_ = m["nbins"]
    rlastbin_ = m["ingest_header"]["task_configuration"]["task_range_info"]["range_last_bin"]/100
    binlength_ = m["ingest_header"]["task_configuration"]["task_range_info"]["step_output_bins"]/100
    horbeamwidth_ = round(m["ingest_header"]["task_configuration"]["task_misc_info"]["horizontal_beam_width"], 2)
    for i in range(10):
        try:
            nrays_expected_ = m["data"][i]["ingest_data_hdrs"]["DB_DBZ"]["number_rays_file_expected"]
            nrays_written_ = m["data"][i]["ingest_data_hdrs"]["DB_DBZ"]["number_rays_file_written"]    
            elevation_ = round(m["data"][i]["ingest_data_hdrs"]["DB_DBZ"]["fixed_angle"], 2)
            break
        except KeyError:
            try:
                nrays_expected_ = m["data"][i]["ingest_data_hdrs"]["DB_DBZ2"]["number_rays_file_expected"]
                nrays_written_ = m["data"][i]["ingest_data_hdrs"]["DB_DBZ2"]["number_rays_file_written"]    
                elevation_ = round(m["data"][i]["ingest_data_hdrs"]["DB_DBZ2"]["fixed_angle"], 2)
                break
            except KeyError:
                continue
    # Append to list
    radarid.append(radarid_)
    dtime.append(dtime_)
    taskname.append(taskname_)
    elevation.append(elevation_)
    nbins.append(nbins_)
    rlastbin.append(rlastbin_)
    binlength.append(binlength_)
    #nrays_expected.append(nrays_expected_)
    #nrays_written.append(nrays_written_)
    fpath.append(f)
    horbeamwidth.append(horbeamwidth_)   

# put attributes in a dataframe
from collections import OrderedDict
df = pd.DataFrame(OrderedDict(
                  {"radarid": radarid,
                   "datetime": dtime,
                   "taskname": taskname,
                   "elevation": elevation,
                   #"nrays_expected": nrays_expected,
                   #"nrays_written": nrays_written,
                   "nbins": nbins,
                   "rlastbin": rlastbin,
                   "binlength": binlength,
                   "horbeamwidth": horbeamwidth,
                   "fpath": fpath                   
                  }))

In [ ]:
# Let's open one scanning mode and one elevation (this will take some minutes to load)
mode = 'VOL_A'
elev = 0.

# Use the dataframe to get the paths that correspond to our selection
paths = df["fpath"].loc[df["elevation"]==elev].loc[df["taskname"]==mode]

In [ ]:
paths = sorted(list(paths))
print(len(paths))


# set engine

In [ ]:
# engine = "netcdf4"
engine = "h5netcdf"

# Reading functions

In [ ]:
def read_single(f):
    reindex = dict(start_angle=-0.5, stop_angle=360, angle_res=1., direction=1)
    ds = xr.open_dataset(f, engine="iris", group="sweep_0", reindex_angle=reindex)
    ds = ds.set_coords("sweep_mode")
    ds = ds.rename_vars(time="rtime")
    ds = ds.assign_coords(time=ds.rtime.min())
    return ds


In [ ]:
@dask.delayed
def process_single(f, num, dest):
    ds = read_single(f)
    moments = [k for k,v in ds.variables.items() if v.ndim == 2]
    new_enc = {k: dwd_enc[k] for k in moments if k in dwd_enc}
    shape = ds[moments[0]].shape
    enc_new = dict(chunksizes=(1, ) + shape[1:])
    [new_enc[k].update(enc_new) for k in new_enc]
    dest = f"{dest}{num:03d}.nc"
    ds.to_netcdf(dest, engine=engine, encoding=new_enc)
    return dest

# Convert Files in subfolder

In [ ]:
%%time
dest = "turkey/test_"
results = []
# fill dask compute pipeline
for i, f in tqdm(enumerate(paths)):
    results.append(client.compute(process_single(f, i, dest)))
# compute pipeline
# this returns, if all results are computed
for res in results:
    print(res.result())    

# Reload converted files

In [ ]:
%%time
dsr = xr.open_mfdataset(f"{dest}*", concat_dim="time", combine="nested", engine=engine)
display(dsr)

# Fix encoding before write to single file

In [ ]:
moments = [k for k,v in dsr.variables.items() if v.ndim == 3]
shape = dsr[moments[0]].shape
enc_new= dict(chunksizes=(1, ) + shape[1:])

drop = ['szip', 'zstd', 'bzip2', 'blosc', 'coordinates']
enc = {k: {key: v.encoding[key] for key in v.encoding if key not in drop} for k, v in dsr.data_vars.items() if k in moments}
[enc[k].update(enc_new) for k in moments if k not in ["DB_HCLASS2"]]
del enc["DB_HCLASS2"]["chunksizes"]
encoding = {k: enc[k] for k in moments}
print(encoding)

# Write to single file

In [ ]:
%%time
dsr.to_netcdf(f"iris-test-compressed-{engine}.nc", engine=engine, encoding=encoding)

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#!h5dump -HBvp iris-test-compressed-h5netcdf.nc